In [110]:
# Import the pathlib and csv library
from pathlib import Path
import csv
# Import numpy libary
import numpy as np
import pandas as pd

#Set the file path
menu_csvpath = Path('./Resources/menu_data.csv')
sale_csvpath = Path('./Resources/sales_data.csv')

# Initialize lists
menu = []
sales = []
item = []
price = []
cost = []
quantity = []
menu_item = []
sales_short = []

# Open menu csv file as the object
with open(menu_csvpath, 'r') as csvfile:
    # Pass in the csv file to the csv.reader() function
    csvreader = csv.reader(csvfile, delimiter=',')
    
    # Read the header row first
    csv_header = next(csvreader)
    
    # Reade each row of data after the header
    for row in csvreader:
        # Append the row data to the list of menu records
        menu.append(row)
        # Append the row item, price, cost to the related lists
        item.append(row[0])
        price.append(row[3])
        cost.append(row[4])

# Open sales csv file as the object
with open(sale_csvpath, 'r') as csvfile:
    # Pass in the csv file to the csv.reader() function
    csvreader = csv.reader(csvfile, delimiter=',')
    
    # Read the header row first
    csv_header = next(csvreader)
    
    # Reade each row of data after the header
    for row in csvreader:
        # Append the row data to the list of menu records
        sales.append(row)
        # Append the row quantity, menu item to the related lists
        quantity.append(row[3])
        menu_item.append(row[4])


menu_dataframe = pd.read_csv(menu_csvpath)
sales_dataframe = pd.read_csv(sale_csvpath)
sales_dataframe.drop(['Line_Item_ID','Date','Credit_Card_Number'], axis=1,inplace = True)
# Initialise report dictionary
report = dict()

# Get items list in sales data set
salesitemset = list(set(menu_item))

# Set item as index of menu dataframe
menu_report = menu_dataframe.set_index(menu_dataframe['item'])

# Loop through menu records to fill dictionary report
for sales_item in salesitemset:
    if sales_item in item:
        item_sindex = sales_dataframe['Menu_Item'].str.contains(sales_item)
        item_count = sales_dataframe['Quantity'].loc[item_sindex].sum()
        item_revenue = menu_report['price'][sales_item]*item_count
        item_cogs = menu_report['cost'][sales_item]*item_count
        item_profit = item_revenue-item_cogs
        report[sales_item]={"01-count": item_count,"02-revenue": item_revenue,"03-cogs": item_cogs,"04-profit": item_profit}
    else:
        print(sales_item + " does not equal items in menu data! NO MATCH!")
        
        
# Set the output file path
output_path = Path('output.txt')

# Open the output path as a file object and output the results in a text file
with open(output_path, 'w') as f:
    for key, value in report.items():  
        f.write('%s:%s\n' % (key, value))
    f.close()

